<a href="https://colab.research.google.com/github/micheldc55/Deep-Learning/blob/main/06_mi_mlp_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
COLAB = True

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1q2shS6fBq_pSI8CBrjzdgg6DTTZfZ2mJ',
                                        dest_path='./funciones_auxiliares.py')
    gdd.download_file_from_google_drive(file_id='1o2ZUFTRmKf9N8Dx4WDzF2fzaYGErpTlF',
                                        dest_path='./datasets/fuga_clientes/fuga_clientes_empresa_telefonica_construccion.csv')

In [ ]:
!ls -la datasets/fuga_clientes

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix

%matplotlib inline

In [ ]:
FUGA_CLIENTES = True

if FUGA_CLIENTES:
    path = './datasets/fuga_clientes/'
    X_df = pd.read_csv(path+'fuga_clientes_empresa_telefonica_construccion.csv')
    X_df.drop('Customer ID', axis=1, inplace=True)
    X = X_df.drop("Churn Status", axis=1).values
    y = X_df["Churn Status"].values
else:
    from sklearn.datasets import load_breast_cancer
    
    a = load_breast_cancer()
    
    X = a.data
    y = a.target
    X_df = pd.DataFrame(columns=a.feature_names, data=X)
    X_df["target"] = y

X_df[:5]

In [ ]:
np.unique(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import seaborn as sns

sns.clustermap(X_df.corr(), vmin=-1, vmax=1, cmap='bwr');

In [ ]:
#X_df.drop('AveBedrms', axis=1, inplace=True)
#X_df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

X_df[:5]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_sc = sc.transform(X_train)
X_test_sc  = sc.transform(X_test)

model = LogisticRegression()
model

In [ ]:
model.fit(X_train_sc, y_train)
y_train_pred = model.predict(X_train_sc)
y_test_pred  = model.predict(X_test_sc)

### Primero estandarizo

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_sc = sc.transform(X_train)
X_test_sc  = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_sc, y_train)
y_train_pred = model.predict(X_train_sc)
y_test_pred  = model.predict(X_test_sc)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test,  y_test_pred))

### MLP implementado a mano

In [ ]:
np.clip([0,-1,2], 0, None)

In [ ]:
def my_model(x, W):
    bI, WI, bH, WH = W
    salida_oculta = np.clip(bI + x.dot(WI), 0, None)
    aux = bH + salida_oculta.dot(WH)
    return 1/(1+np.exp(-aux))

In [ ]:
Noc = 2
w = np.array([np.random.randn(Noc),
              np.random.randn(X.shape[1], Noc),
              np.random.randn(1),
              np.random.randn(Noc)], dtype=object)
w

In [ ]:
print(X_train[0].shape)
print(w.shape)

In [ ]:
my_model(X_train_sc[0:10], w)

In [ ]:
from sklearn.metrics import log_loss

def mi_funcion_error(data, w):
    eps = 1e-20
    X, y = data
    ypred = my_model(X, w)
    return (-y*np.log(eps+ypred) - (1-y)*np.log(eps+(1-ypred))).mean() + 0.001*(w**2).sum().sum()


def mi_funcion_error2(data, w):
    X, y = data
    ypred = my_model(X, w)
    return log_loss(y, ypred)


In [ ]:
def calcula_grad(funcion, data, W, eps=1e-4):
    fw = funcion(data, W)
    bI, WI, bH, WH = W.copy()
    grad_bI = 0*bI
    grad_WI = 0*WI
    grad_bH = 0*bH
    grad_WH = 0*WH
    
    # perturbaciones en bI
    for i in range(len(bI)):
        a = bI[i]
        bI[i] += eps
        fw2 = funcion(data, np.array([bI, WI, bH, WH], dtype=object))
        grad_bI[i] = (fw2 - fw) / eps
        bI[i] = a
    
    # perturbaciones en WI
    for i in range(WI.shape[0]):
        for j in range(WI.shape[1]):
            a = WI[i,j]
            WI[i,j] += eps
            fw2 = funcion(data, np.array([bI, WI, bH, WH], dtype=object))
            grad_WI[i,j] = (fw2 - fw) / eps
            WI[i,j] = a
    
    # perturbaciones en bH
    a = bH[0]
    bH += eps
    fw2 = funcion(data, np.array([bI, WI, bH, WH], dtype=object))
    grad_bH[0] = (fw2 - fw) / eps
    bH[0] = a

    # perturbaciones en WH
    for i in range(len(WH)):
        a = WH[i]
        WH[i] += eps
        fw2 = funcion(data, np.array([bI, WI, bH, WH], dtype=object))
        grad_WH[i] = (fw2 - fw) / eps
        WH[i] = a
    
    return np.array([grad_bI, grad_WI, grad_bH, grad_WH], dtype=object)

In [ ]:
calcula_grad(my_model, X_train_sc[:1], w)

In [ ]:
X_train_sc[0]

In [ ]:
X_train_sc[1]

In [ ]:
w

In [ ]:
#mi_funcion_error(my_model, (X_train_sc[:10], y_train_sc[:10]), w)
mi_funcion_error((X_train_sc[:10], y_train[:10]), w)

In [ ]:
(w**2).sum().sum()

In [ ]:
calcula_grad(mi_funcion_error, (X_train_sc[0:10], y_train[0:10]), w)

### Entrenamiento con batch

In [ ]:
alpha = 2
nepocas = 200
batch_size = 20
Ntr = len(X_train_sc)

Noc = 2*5
w = np.array([np.random.randn(Noc),
              np.random.randn(X.shape[1], Noc),
              np.random.randn(1),
              np.random.randn(Noc)], dtype=object)

errores_tr = []
alpha_norm = alpha/batch_size

for e in range(nepocas):
    for b in range(0, Ntr, batch_size):
        X_lote = X_train_sc[b:(b+batch_size)]
        y_lote = y_train   [b:(b+batch_size)]
        grad = calcula_grad(mi_funcion_error, (X_lote,y_lote), w)
        w -= alpha_norm*grad
        
    error_total_tr = mi_funcion_error((X_train_sc,y_train), w)
    print("Error en training en época {}: {}".format(e, error_total_tr))
    errores_tr.append(error_total_tr)

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(errores_tr, label='error_total_tr')
plt.xlabel("época", fontsize=14)
plt.ylabel("error", fontsize=14)
plt.legend();

In [ ]:
y_tr_preds = 1*(my_model(X_train_sc, w)>0.5)
y_te_preds = 1*(my_model(X_test_sc, w)>0.5)

y_tr_preds

In [ ]:
print(accuracy_score(y_train, y_tr_preds))
print(accuracy_score(y_test,  y_te_preds))